In [3]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [2]:
all_commits = pd.DataFrame()
all_commits_query = f"""
        SELECT DISTINCT(commits.cmt_commit_hash), repo.repo_id, repo.repo_name, repo.repo_path, 
            commits.cmt_author_email, commits.cmt_author_timestamp from repo, commits
        WHERE 
            repo.repo_id = commits.repo_id
        GROUP BY repo.repo_id, commits.cmt_commit_hash, commits.cmt_author_email, commits.cmt_author_timestamp
        ORDER BY repo.repo_id;
        """
all_commits = pd.read_sql_query(all_commits_query, con=engine)
all_commits

,cmt_commit_hash,repo_id,repo_name,repo_path,cmt_author_email,cmt_author_timestamp
0,0097eea9b9cfa0605916fac225513c14a808c910,25430,rmq-0mq,github.com/rabbitmq/,sustrik@250bpm.com,2010-09-09 08:49:09+00:00
1,04b6ed16843b336ce6ab18a693c94bbaf669dcee,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-08-13 11:35:42+00:00
2,0a43d1cb58bab9162f50bd4c75bed5bd17bd51ae,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-11-13 18:14:44+00:00
3,0e937a3370247b8f65e7a3eed0f66aea7f255453,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-09-15 11:26:31+00:00
4,1436532e5e2e7381e184b52356dfb3dc3740b636,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-09-10 16:56:12+00:00
...,...,...,...,...,...,...
1657055,dab5ef1ceb6c587718cdaf2e46fd47f5d7958190,30332,flask-session,github.com/carbonblack/,fsp261@gmail.com,2015-09-17 05:20:15+00:00
1657056,dc7c5ae16d8fd00584860ee322b68d5b0c0a01dc,30332,flask-session,github.com/carbonblack/,fsp261@gmail.com,2014-04-25 11:48:29+00:00
1657057,e2b18855384c62d50cc00faf37c7d77c968c2f86,30332,flask-session,github.com/carbonblack/,fsp261@gmail.com,2014-04-29 12:02:27+00:00
1657058,e2f676655a7b6669cf5a553a6682fcf5a1a3bda0,30332,flask-session,github.com/carbonblack/,skroh@carbonblack.com,2020-04-09 20:27:03+00:00


In [3]:
    by_repo = all_commits.loc[all_commits.groupby('repo_id').cmt_author_timestamp.idxmax()].sort_values('cmt_author_timestamp')
    
    import datetime

    current = datetime.date.today()
    start = current - datetime.timedelta(days=180)
    print(start)


2019-10-19


In [6]:
# This gives us the top contribs per repo - this can be printed to csv file 
i=0
for index, row in by_repo.iterrows():
    repo_link = row.repo_path + row.repo_name
    org = row.repo_path.split("/")[1]
    print(repo_link, org)
    #print(row.repo_path, row.repo_name, row.cmt_author_timestamp, row.cmt_author_email)
    #print(type(row.cmt_author_timestamp))
    top_contribs = all_commits.loc[all_commits['repo_id'] == row.repo_id].cmt_author_email.value_counts()
    if len(top_contribs) > 1:
        print(top_contribs.index[0], top_contribs[0], top_contribs.index[1], top_contribs[1])
    elif len(top_contribs) == 1:
        print(top_contribs.index[0], top_contribs[0])
    elif len(top_contribs) == 0:
        print('None')
    i+=1
    if i == 50:
        break

github.com/rabbitmq/rabbithub rabbitmq
tonyg@rabbitmq.com 80 mikeb@squaremobius.net 2
github.com/vmware/pyvco vmware
yann.hodique@gmail.com 50
github.com/rabbitmq/rabbitmq-ha rabbitmq
matthew@lshift.net 118
github.com/Spring-projects/cloudfoundry-sandbox Spring-projects
peter@cacoethes.co.uk 1 mark@josquin.local 1
github.com/Spring-projects/protocol-fallback-demo Spring-projects
ozhurakousky@pivotal.io 12 bruce.snyder@gmail.com 3
github.com/Spring-projects/nio-sandbox Spring-projects
jon@jbrisbin.com 11
github.com/rabbitmq/rabbitmq-heroku-example rabbitmq
dpw@lshift.net 2 majek@lshift.net 2
github.com/Spring-projects/spring-build Spring-projects
nebhale@nebhale.com 29
github.com/Spring-projects/spring-net-threading Spring-projects
kxu@bn.com 85 gcaprio@1530technologies.com 51
github.com/Spring-projects/insight-plugin-mongodb Spring-projects
stephen@harrison.org 15 jkew@vmware.com 8
github.com/Spring-projects/spring-gemfire-net Spring-projects
mpollack@pivotal.io 39 mpollack@gopivotal.c

# Top Committers by Org

In [9]:
all_commits_query = f"""
        SELECT DISTINCT(commits.cmt_commit_hash), repo.repo_id, repo.repo_group_id, repo.repo_name, repo.repo_path, 
            commits.cmt_author_email, commits.cmt_author_timestamp from repo, commits
        WHERE 
            repo.repo_id = commits.repo_id
            AND commits.cmt_author_email NOT LIKE 'containers%%'
            AND commits.cmt_author_email NOT LIKE 'k8s%%'
            AND commits.cmt_author_email NOT LIKE 'cf-%%'
            AND commits.cmt_author_email NOT LIKE 'ci@%%'
            AND commits.cmt_author_email NOT LIKE 'cb@%%'
            AND commits.cmt_author_email NOT LIKE '%%cve@%%'
            AND commits.cmt_author_email NOT LIKE '%%cves@%%'
        GROUP BY repo.repo_id, commits.cmt_commit_hash, commits.cmt_author_email, commits.cmt_author_timestamp
        ORDER BY repo.repo_id;
        """
all_commits = pd.read_sql_query(all_commits_query, con=engine)
by_repo = all_commits.loc[all_commits.groupby('repo_id').cmt_author_timestamp.idxmax()].sort_values('cmt_author_timestamp')

In [10]:
by_org = all_commits.loc[all_commits.groupby('repo_group_id').cmt_author_timestamp.idxmax()].sort_values('cmt_author_timestamp')
for index, row in by_org.iterrows():
    print 
    top_contribs = all_commits.loc[all_commits['repo_group_id'] == row.repo_group_id].cmt_author_email.value_counts()
    print(row.repo_path, top_contribs.index[0], top_contribs[0], top_contribs.index[1], top_contribs[1], top_contribs.index[2], top_contribs[2])

github.com/GemXD/ swale@snappydata.io 10 gmlewis@google.com 1 ashetkar@users.noreply.github.com 1
github.com/pcfdev-forks/ suraci.alex@gmail.com 4908 justin@specialbusservice.com 3074 crosbymichael@gmail.com 2013
github.com/cfmobile/ david_syer@hotmail.com 921 hone02@gmail.com 723 krishicks@gmail.com 656
github.com/pivotal-education/ droberts@gopivotal.com 110 scottyfred@gmail.com 78 amcginlay@users.noreply.github.com 77
github.com/Pivotal-DataFabric/ jpatel-pivotal@pivotal.io 255 jgronline@gmail.com 171 @Vengers12 146
github.com/pivotal-gss/ bruce@momjian.us 8781 tgl@sss.pgh.pa.us 7844 heikki.linnakangas@iki.fi 1890
github.com/pivotalsoftware/ haoyuan@cs.berkeley.edu 1073 Florian.Schoppmann@emc.com 550 ilayaperumalg@gmail.com 526
github.com/pivotaltracker/ KrauseFx@gmail.com 8712 oliver@cocoanetics.com 2225 jeresig@Archimedes.local 1714
github.com/Pivotal-sg/ raymond.penners@intenct.nl 846 asim@chuhnk.me 224 mcheng.work@gmail.com 207
github.com/Pivotal-Field-Engineering/ bruce@momjian